In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

В этом ноутбуке вы познакомитесь с основными средствами работы с пакетом pandas.
Задачей на сегодня является научиться работать с pandas в рамках описательной статистики.
Пакет пандас импортирован в самом начале "import pandas as pd"

In [ ]:
# Here we are reading csv files from the sources
data = pd.read_csv("/kaggle/input/imdb-extensive-dataset/IMDb movies.csv")
rating = pd.read_csv("/kaggle/input/imdb-extensive-dataset/IMDb ratings.csv")
actors = pd.read_csv("/kaggle/input/imdb-extensive-dataset/IMDb names.csv")
titles = pd.read_csv("/kaggle/input/imdb-extensive-dataset/IMDb title_principals.csv")

# to view columns of the panel table we just print the "columns" field of the object "dataframe".
# The latter "dataframe" is the core class in pandas.

print(actors.columns)
print(titles.columns)
print(rating.columns)
print(data.columns)

In [ ]:
# here we concretize te features that are interesing for us
features = ['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics']
for feature in features:
    print(feature, data[feature].unique()) # unique method gives all unique values that are in the column "feature". 

In [ ]:
def get_currecy(x):
    if len(str(x).split())>1:
        return str(x).split()[0]
    else:
        return "$"

def get_amount(x):
    if len(str(x).split())>1:
        return float(str(x).split()[1])
    else:
        return float(x)
    
# we have defined functions above and want to apply them to some column
# to apply we use .apply(function) method. 

data["budget currecy"] = data["budget"].apply(get_currecy)
data["budget amount"] = data["budget"].apply(get_amount)

# we do not need the "budget" column itself, so we can delete it from the dataframe

del data["budget"]

data["worlwide_gross_income currecy"] = data["worlwide_gross_income"].apply(get_currecy)
data["worlwide_gross_income amount"] = data["worlwide_gross_income"].apply(get_amount)
del data["worlwide_gross_income"]

data["usa_gross_income currecy"] = data["usa_gross_income"].apply(get_currecy)
data["usa_gross_income amount"] = data["usa_gross_income"].apply(get_amount)
del data["usa_gross_income"]



money = ["budget currecy", "budget amount", 
         "usa_gross_income currecy", "usa_gross_income amount", 
         "worlwide_gross_income currecy", "worlwide_gross_income amount"]

# here we want to filter the dataframe by some condition, for example to filter the records with dollar currency
# it is made using the condition data[condition].

new_data = data[money][(data["budget currecy"] == "$") & 
                       (data["usa_gross_income currecy"] == "$") & 
                       (data["worlwide_gross_income currecy"] == "$")].dropna()

# dropna is used to drop all records including NaN values 

new_data["usa_income/budget ratio"] = new_data["usa_gross_income amount"].apply(float)/new_data["budget amount"].apply(float)
new_data["world_income/budget ratio"] = new_data["worlwide_gross_income amount"].apply(float)/new_data["budget amount"].apply(float)

data = data.merge(new_data)
col_names = data.columns
for name in col_names:
    if "currecy" in name:
        data = data[data[name]=="$"]
        del data[name]

data["duration"] = data["duration"].apply(float)

In [ ]:
#here we buila a histogram

data["world_income/budget ratio"].hist()

In [ ]:
print(data[data["world_income/budget ratio"]>500])

In [ ]:
data = data[data["world_income/budget ratio"]<100]

In [ ]:
data["world_income/budget ratio"].hist()

In [ ]:
from math import log

data["log(usa_income/budget ratio)"] = data["usa_income/budget ratio"].apply(log)
data["log(usa_income/budget ratio)"].hist()
data["log(usa_income/budget ratio)"].describe()

In [ ]:
data["log(world_income/budget ratio)"] = data["world_income/budget ratio"].apply(log)
data["log(world_income/budget ratio)"].hist()
data["log(world_income/budget ratio)"].describe()

In [ ]:
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt

ecdf = ECDF(data["world_income/budget ratio"])
plt.plot(ecdf.x,ecdf.y, label = "Эмпирическая функция распределения")
plt.plot(ecdf.x*0+1,ecdf.y, color="black")
plt.plot(ecdf.x,ecdf.y*0+0.42, color="black")
plt.show()

In [ ]:
dynamics_mean = data.groupby("year").mean()
dynamics_std = data.groupby("year").std()

# maplotlib is added to gain visual aids
from matplotlib import pyplot as plt

plt.plot(dynamics_mean["world_income/budget ratio"], label="средняя эффективность")
plt.plot(dynamics_mean["log(usa_income/budget ratio)"]*0, color="black")
plt.legend() #we add legend to the plot
plt.show()
plt.plot(dynamics_mean["world_income/budget ratio"]*0, color="black")
plt.plot(dynamics_std["world_income/budget ratio"], label="ср.кв. откл. эффективности")
plt.legend()
plt.show()
plt.plot(dynamics_mean["log(world_income/budget ratio)"], label="средняя логарифмическая эффективность")
plt.plot(dynamics_mean["log(usa_income/budget ratio)"]*0, color="black")
plt.legend()
plt.show()
plt.plot(dynamics_mean["world_income/budget ratio"]*0, color="black")
plt.plot(dynamics_std["log(world_income/budget ratio)"], label="ср.кв. откл. логарифмической эффективности")
plt.legend()
plt.show()


In [ ]:
dynamics_mean = data.groupby("year").mean()
dynamics_std = data.groupby("year").std()

from matplotlib import pyplot as plt

plt.scatter(data["year"], data["log(world_income/budget ratio)"])
plt.plot(dynamics_mean["log(usa_income/budget ratio)"]*0, color="black")
plt.legend()
plt.show()
plt.scatter(data["year"], data["world_income/budget ratio"])
plt.plot(dynamics_mean["log(usa_income/budget ratio)"]*0+1, color="black")
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot as plt


plt.scatter(data["year"], data["log(world_income/budget ratio)"])
plt.plot(dynamics_mean["log(usa_income/budget ratio)"]*0, color="black")
plt.legend()
plt.show()

In [ ]:
data.corr()

In [ ]:
def collect_set(df_series, col_name):
    answer = set()
    for value in df_series[col_name].apply(lambda x: str(x).split(",")):
        for x in value:
            x = x.strip()
            if not (x in answer):
                answer.add(x)
    return answer

genres = collect_set(data, "genre")
countries = collect_set(data, "country")
languages = collect_set(data, "language")

def splitter(x):
    x = str(x).split(",")
    mt = []
    for word in x:
        mt.append(word.strip())
    return mt

print("Genres")
for genre in genres:
    length = len(data[data["genre"].apply(lambda x: int(genre in splitter(x)))==1])
    if length>100:
        print(genre, length)
print()
print("Countries")
for country in countries:
    length = len(data[data["country"].apply(lambda x: int(country in splitter(x)))==1])
    if length>100:
        print(country, length)
print()
print("Languages")
for language in languages:
    length = len(data[data["language"].apply(lambda x: int(language in splitter(x)))==1])
    if length>100:
        print(language, length)


In [ ]:
Languages = ["Spanish", "Russian", "Arabic", "Mandarin", "English", "French", "Italian", "Japanese", "German"]
Rus = ["Испанский", "Русский", "Арабский", "Китайский", 
       "Английский", "Французский", "Итальянский", "Японский", "Немецкий"]
Rus_Languages=dict()
for i in range(len(Languages)):
    Rus_Genres[Languages[i]] = Rus[i]
print(Rus_Genres)


for language in Languages:
    loc_data = data[data["language"].apply(lambda x: language in splitter(x))]
    #if (loc_data1["world_income/budget ratio"].mean()<standard_deviation):
    #    loc_data1 = loc_data1[loc_data1["world_income/budget ratio"]<standard_deviation]
    #    standard_deviation = loc_data1["world_income/budget ratio"].std()
    #if (loc_data2["usa_income/budget ratio"].mean()<standard_deviation):
    #    loc_data1 = loc_data2[loc_data2["usa_income/budget ratio"]<standard_deviation]
    #    standard_deviation = loc_data2["usa_income/budget ratio"].std()
    dynamics_mean = loc_data.groupby("year").mean()
    dynamics_std = loc_data.groupby("year").std()
    print(language, "World", loc_data["log(world_income/budget ratio)"].mean(), loc_data["log(world_income/budget ratio)"].std())
    plt.ylim(top=3)
    plt.ylim(bottom=-3)
    plt.plot(dynamics_mean["log(world_income/budget ratio)"], label=Rus_Genres[language])
    plt.plot(dynamics_mean["log(world_income/budget ratio)"]*0, color="black")
    plt.legend()
    plt.show()


In [ ]:
Countries = ["Spain", "UK", "France", "USA", "Australia", "Mexico", "China", "Japan", "Germany", "Hong Kong",
             "Canada", "Italy"]
Rus = ["Испания", "Великобритания", "Франция", "США",  "Австралия",
       "Мексика", "Китай", "Япония", "Германия", "Гонконг", "Канада", "Италия"]
Rus_Countries=dict()
for i in range(len(Countries)):
    Rus_Countries[Countries[i]] = Rus[i]
print(Rus_Countries)

for country in Countries:
    loc_data = data[data["country"].apply(lambda x: country in splitter(x))]
    dynamics_mean = loc_data.groupby("year").mean()
    dynamics_std = loc_data.groupby("year").std()
    print(country, "World", loc_data["log(world_income/budget ratio)"].mean(), loc_data["log(world_income/budget ratio)"].std())
    plt.plot(dynamics_mean["log(world_income/budget ratio)"], label=Rus_Countries[country])
    plt.plot(0*dynamics_mean["log(usa_income/budget ratio)"])
    plt.legend()
    plt.show()


In [ ]:
from math import floor
from scipy.special import softmax

def count_rating_max(x):
    answer = np.dot(np.array(x), softmax(x))
    return answer

def count_rating(x):
    answer = np.mean(x)
    return answer

def count_rating_min(x):
    answer = np.dot(np.array(x), softmax(-np.array(x)))
    return answer


new_directors_df = directors_df.groupby("Director_for_grouping")
new_directors_df = new_directors_df.agg(lambda x: list(x))
new_directors_df = new_directors_df.reset_index()
new_directors_df["rating mean"] = new_directors_df["log(world_income/budget ratio)"].apply(count_rating)
new_directors_df["rating mean"].hist()
for i in range(len(new_directors_df)):
    X = new_directors_df["year"].loc[i]
    Y = [new_directors_df["rating mean"].loc[i]]*len(X)
    plt.scatter(X,Y)
plt.plot([0]*len(data["year"]))
plt.xlim(left=1980)
plt.xlim(right=2020)
plt.ylim(bottom=-5)
plt.ylim(top=5)
plt.show()

In [ ]:
features = pd.DataFrame()
Producer = collect_set(data, "production_company")
for production_company in Producer:
    prod = pd.DataFrame(data["production_company"].apply(lambda x: production_company in splitter(x)))
    prod[production_company] = prod["production_company"] 
    del prod["production_company"] 
    #loc_data = data[prod]
    features = pd.concat([features, prod], axis=1)

In [ ]:
Writers = collect_set(data, "writer")
for writer in Writers:
    prod = pd.DataFrame(data["writer"].apply(lambda x: writer in splitter(x)))
    prod[writer] = prod["writer"] 
    del prod["writer"] 
    #loc_data = data[prod]
    features = pd.concat([features, prod], axis=1)

In [ ]:
Directors = collect_set(data, "director")
for director in Directors:
    direc = pd.DataFrame(data["director"].apply(lambda x: director in splitter(x)))
    direc[director] = direc["director"] 
    del direc["director"] 
    #loc_data = data[prod]
    features = pd.concat([features, direc], axis=1)

In [ ]:
features.to_csv("boolean.csv")

In [ ]:
features = features.applymap(lambda x: int(x))

In [ ]:
features.to_csv("boolean_maped.csv")

In [ ]:
target = data["log(world_income/budget ratio)"]

In [ ]:
from math import sqrt

def criteria(x):
    if x==1:
        return True
    else:
        return x/sqrt(1-x*x)*sqrt(l-2)>1.96

backup = data.copy()
l = len(data)
del data["log(usa_income/budget ratio)"]
del data["usa_income/budget ratio"]
del data["usa_gross_income amount"]
del data["reviews_from_users"]
del data["reviews_from_critics"]
del data["metascore"]
del data["votes"]
del data["avg_vote"]
cor = data.corr()
cor

In [ ]:
features = pd.concat([features, data["budget amount"]], axis=1)
features = pd.concat([features, data["duration"]], axis=1)

In [ ]:
#del features["budget amount"]
del features["duration"]

In [ ]:
dataframe = features.corrwith(target).apply(criteria)
for i in dataframe[dataframe].index:
    print(i, dataframe.loc[i])

In [ ]:
from math import exp

output_df = pd.DataFrame(features.columns)
output_df["factor"] = output_df[0]
del output_df[0]
output_df = pd.concat([output_df, pd.DataFrame(model.coef_)], axis=1)
output_df["coef"] = output_df[0]
del output_df[0]
output_df["a's"] = output_df["coef"].apply(exp)
print(output_df)

In [ ]:
len(data)

In [ ]:
new_features = features[dataframe[dataframe].index]
new_features = pd.concat([new_features, data["budget amount"]], axis=1)

In [ ]:
from math import exp

output_df = pd.DataFrame(new_features.columns)
output_df["factor"] = output_df[0]
del output_df[0]
output_df = pd.concat([output_df, pd.DataFrame(model.coef_)], axis=1)
output_df["coef"] = output_df[0]
del output_df[0]
output_df["a's"] = output_df["coef"].apply(exp)
print(output_df)

In [ ]:
for i in output_df.sort_values(by="coef").index:
    print(i, output_df["factor"].loc[i], output_df["a's"].loc[i])

In [ ]:
Actors = collect_set(data, "actors")

In [ ]:
l = len(Actors)
i = 0
for actors in Actors:
    act = pd.DataFrame(data["actors"].apply(lambda x: actors in splitter(x)))
    act[actors] = act["actors"] 
    del act["actors"] 
    act = act.corrwith(target).apply(criteria).loc[actors]
    if act:
        print(actors, ";", act, ";", i, l)
    i+=1

In [ ]:
influencial_actors = pd.read_csv("/kaggle/input/inf-actors/IMDB_Influencial_actors.csv", sep=";", header=None)
print(influencial_actors)

In [ ]:
features_2 = pd.DataFrame()
i=0
for actor in list(influencial_actors[0]):
    actors = pd.DataFrame(data["actors"].apply(lambda x: actor in splitter(x)))
    actors[actor] = actors["actors"]
    del actors["actors"]
    features_2 = pd.concat([features_2, actors], axis=1)
    print(i, end="; ")
    i += 1
    if i%20==0:
        print()
print(features_2)
    

In [ ]:
features_2.to_csv("boolean2.csv")

In [ ]:
features_2.applymap(lambda x: int(x)).to_csv("boolean2_maped.csv")

In [ ]:
features_2 = pd.read_csv("boolean2_maped.csv")

In [ ]:
features_genre = pd.DataFrame()
for genre in Genres:
    features_genre[genre] = data["genre"].apply(lambda x: genre in splitter(x))
features_genre = features_genre.applymap(lambda x: int(x))

In [ ]:
from math import exp

output_df = pd.DataFrame(features_3.columns)
output_df["factor"] = output_df[0]
del output_df[0]
output_df = pd.concat([output_df, pd.DataFrame(model.coef_)], axis=1)
output_df["coef"] = output_df[0]
del output_df[0]
output_df["a's"] = output_df["coef"].apply(exp)
print(output_df)

In [ ]:
print(output_df.loc[12186:12204])

In [ ]:
for genre in Genres:
    print(genre, (target-target.mean() - 0.163059*features_3[genre]).apply(lambda x: x**2).sum()/target.apply(lambda x: x**2).sum())


In [ ]:
from nltk.corpus import stopwords #we must reduce the stopwords from the text

data["description"] = data["description"].apply(lambda x: str(x).lower())

print("1 OK!")

stopWords = set(stopwords.words('english')) 

for sep in [".", ",", "!", ")", "(", "&", "*", "_", "-", "+", "=", "\'", "\"", "|", ":", ";", "/"]:
    data["description"] = data["description"].apply(lambda x: x.replace(sep, " "))

print("2 OK!")
    
def trim(x):
    while "  " in x:
        x = x.replace("  ", " ")
        print(x)
    return x

print("3 OK!")

data["description"] = data["description"].apply(trim)

print("4 OK!")

for w in stopWords:
    data["description"] = data["description"].apply(lambda x: x.replace(" "+w+" ", " "))

print("5 OK!")
    
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

print("6 OK!")

tfidf_train = vectorizer.fit_transform(data["description"])

print("7 OK!")

from sklearn.decomposition import PCA
pca = PCA(n_components=500)
lower_dim_tfidf_train = pca.fit_transform(tfidf_train.todense())

In [ ]:
lower_dim_tfidf_train

# Упражнения

1. Сформируйте many-to-many таблицу, в которой можно посмотреть пары режиссер, актер и количество общих работ

2. Сформируйте many-to-many, где для любых двух режиссеров установлено число "общих" актеров - сколько актеров работало и у того, и у того режиссеров